In [35]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Configuration d'affichage
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')
sns.set_palette("crest")

In [36]:
# --- 1. Chargement du jeu de données et nettoyage des noms ---

try:
    df = pd.read_csv("autos.csv", encoding='latin1')
except FileNotFoundError:
    print("Erreur : Le fichier 'autos.csv' est introuvable. Veuillez le placer dans le répertoire courant.")
    exit()

# Nettoyage des noms des colonnes
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

print(f"Jeu de données chargé : {df.shape[0]} lignes et {df.shape[1]} colonnes")
print("\n🔎 Informations initiales :")
df.info()

Jeu de données chargé : 371528 lignes et 21 colonnes

🔎 Informations initiales :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   index                371528 non-null  int64 
 1   datecrawled          371528 non-null  object
 2   name                 371528 non-null  object
 3   seller               371528 non-null  object
 4   offertype            371528 non-null  object
 5   price                371528 non-null  int64 
 6   abtest               371528 non-null  object
 7   vehicletype          333659 non-null  object
 8   yearofregistration   371528 non-null  int64 
 9   gearbox              351319 non-null  object
 10  powerps              371528 non-null  int64 
 11  model                351044 non-null  object
 12  kilometer            371528 non-null  int64 
 13  monthofregistration  371528 non-null  int64 
 14  fue

In [37]:
# --- 2. Suppression des colonnes non pertinentes ---

colonnes_a_supprimer = [
    'index', 
    'datecrawled', 
    'name', 
    'seller', 
    'offertype', 
    'abtest', 
    'datecreated', 
    'nrofpictures', 
    'postalcode', 
    'lastseen'
]

# Suppression des colonnes
df.drop(columns=colonnes_a_supprimer, inplace=True)

print(f"\n✅ Colonnes non pertinentes supprimées : {colonnes_a_supprimer}")
print(f"Colonnes restantes : {df.columns.tolist()}")


✅ Colonnes non pertinentes supprimées : ['index', 'datecrawled', 'name', 'seller', 'offertype', 'abtest', 'datecreated', 'nrofpictures', 'postalcode', 'lastseen']
Colonnes restantes : ['price', 'vehicletype', 'yearofregistration', 'gearbox', 'powerps', 'model', 'kilometer', 'monthofregistration', 'fueltype', 'brand', 'notrepaireddamage']


In [38]:
# --- 3. Traitement des Doublons ---

nb_doublons = df.duplicated().sum()
df = df.drop_duplicates()
print(f"\nNombre de doublons supprimés : {nb_doublons}")
print(f"Nouvelle taille du dataset : {df.shape}")


Nombre de doublons supprimés : 28503
Nouvelle taille du dataset : (343025, 11)


In [39]:
# --- 4. Traitement des Doublons ---

lignes_doublons = df.duplicated().sum()
df = df.drop_duplicates()
print(f"Nombre de doublons supprimés : {lignes_doublons}. Taille actuelle : {df.shape}")




Nombre de doublons supprimés : 0. Taille actuelle : (343025, 11)


In [40]:
# --- 5. Traitement des valeurs manquantes (Suppression + Imputation simple) ---

# a) Suppression des lignes sans information essentielle (Modèle et Boîte de Vitesse)
# On choisit de supprimer les NaNs pour les features les plus critiques.
lignes_avant_drop = df.shape[0]
df = df.dropna(subset=['model', 'gearbox'])
lignes_supprimees_essentiel = lignes_avant_drop - df.shape[0]

print(f"\nLignes sans 'model' ou 'gearbox' supprimées : {lignes_supprimees_essentiel}")


Lignes sans 'model' ou 'gearbox' supprimées : 33604


In [41]:
# --- 6. Traitement des valeurs aberrantes (Outliers) ---

lignes_avant_outlier = df.shape[0]

# a) Filtrage du prix (variable cible)
# On supprime les prix <= 100 (non significatifs, erreurs ou prix symboliques)
df = df[df['price'] > 100]

# b) Filtrage de l'année d'enregistrement
# On garde les années réalistes (entre 1950 et 2025)
df = df[(df['yearofregistration'] >= 1950) & (df['yearofregistration'] <= 2025)]

# c) Filtrage de la puissance moteur (powerps)
# On supprime les valeurs non réalistes ou extrêmes (Puissance entre 50 PS et 500 PS)
df = df[(df['powerps'] >= 50) & (df['powerps'] <= 500)].copy()

lignes_supprimees_outlier = lignes_avant_outlier - df.shape[0]

print(f"\nLignes avec valeurs aberrantes (prix, année, puissance) supprimées : {lignes_supprimees_outlier}")


Lignes avec valeurs aberrantes (prix, année, puissance) supprimées : 34576


In [42]:
print("\n--- Résumé du nettoyage de données ---")
print(f"Dimensions finales du dataset : {df.shape}")
print(f"Colonnes finales : {df.columns.tolist()}")
print("\nStatistiques des variables numériques restantes (pour vérification):")
print(df.describe().T)

print("\nLe jeu de données est maintenant nettoyé et prêt pour l'analyse.")


--- Résumé du nettoyage de données ---
Dimensions finales du dataset : (274845, 11)
Colonnes finales : ['price', 'vehicletype', 'yearofregistration', 'gearbox', 'powerps', 'model', 'kilometer', 'monthofregistration', 'fueltype', 'brand', 'notrepaireddamage']

Statistiques des variables numériques restantes (pour vérification):
                        count      mean       std     min       25%       50%  \
price               274845.00  10043.15 550302.17  101.00   1500.00   3550.00   
yearofregistration  274845.00   2003.82      6.79 1950.00   1999.00   2004.00   
powerps             274845.00    127.37     58.96   50.00     84.00    116.00   
kilometer           274845.00 125518.57  39127.90 5000.00 100000.00 150000.00   
monthofregistration 274845.00      6.03      3.56    0.00      3.00      6.00   

                          75%         max  
price                 8000.00 99999999.00  
yearofregistration    2008.00     2019.00  
powerps                150.00      500.00  
kilomet

# Featuring
---
## 1. Imports et Configuration

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn')
sns.set_palette("crest")
%matplotlib inline

# Configuration pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("Imports réussis")

Imports réussis


---
## 2. Chargement des Données Nettoyées

In [44]:
# Charger les données nettoyées (output du notebook de data cleaning)
# Note: Assurez-vous que le notebook de data cleaning a été exécuté et a sauvegardé le fichier
# df = pd.read_csv('autos_cleaned.csv', encoding='latin1')

print("="*70)
print("CHARGEMENT DES DONNÉES NETTOYÉES")
print("="*70)
print(f"\nShape du dataset: {df.shape}")
print(f"Lignes: {df.shape[0]:,}")
print(f"Colonnes: {df.shape[1]}")
print(f"\nValeurs manquantes totales: {df.isnull().sum().sum()}")

CHARGEMENT DES DONNÉES NETTOYÉES

Shape du dataset: (274845, 11)
Lignes: 274,845
Colonnes: 11

Valeurs manquantes totales: 67241


In [45]:
# Aperçu des données
print("\nAperçu des premières lignes:")
display(df.head())


Aperçu des premières lignes:


,price,vehicletype,yearofregistration,gearbox,powerps,model,kilometer,monthofregistration,fueltype,brand,notrepaireddamage
2,9800,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,1500,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
5,650,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja
6,2200,cabrio,2004,manuell,109,2_reihe,150000,8,benzin,peugeot,nein


In [46]:
# Informations sur les colonnes
print("\nInformations sur les colonnes:")
print(df.info())


Informations sur les colonnes:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 274845 entries, 2 to 371527
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   price                274845 non-null  int64 
 1   vehicletype          259113 non-null  object
 2   yearofregistration   274845 non-null  int64 
 3   gearbox              274845 non-null  object
 4   powerps              274845 non-null  int64 
 5   model                274845 non-null  object
 6   kilometer            274845 non-null  int64 
 7   monthofregistration  274845 non-null  int64 
 8   fueltype             260605 non-null  object
 9   brand                274845 non-null  object
 10  notrepaireddamage    237576 non-null  object
dtypes: int64(5), object(6)
memory usage: 25.2+ MB
None


In [47]:
# Statistiques descriptives
print("\nStatistiques descriptives:")
display(df.describe())


Statistiques descriptives:


,price,yearofregistration,powerps,kilometer,monthofregistration
count,274845.00,274845.00,274845.00,274845.00,274845.00
mean,10043.15,2003.82,127.37,125518.57,6.03
std,550302.17,6.79,58.96,39127.90,3.56
min,101.00,1950.00,50.00,5000.00,0.00
25%,1500.00,1999.00,84.00,100000.00,3.00
50%,3550.00,2004.00,116.00,150000.00,6.00
75%,8000.00,2008.00,150.00,150000.00,9.00
max,99999999.00,2019.00,500.00,150000.00,12.00
